In [1]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder

/Users/reynier/.pyenv/versions/3.9.13/envs/tfod/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/reynier/.pyenv/versions/3.9.13/envs/tfod/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/reynier/.pyenv/versions/3.9.13/envs/tfod/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/reynier/.pyenv/versions/3.9.13/envs/tfod/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/reynier/.pyenv/versions/3.9.13/envs/tfod/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
cau

In [2]:
path_to_model = '../../../downloaded_models_trinity/231006_01_digits+capitals_efficientnet_799x504/'

In [3]:
# Replace this with your pipeline config path and checkpoint path
pipeline_config_path = os.path.join(path_to_model,'pipeline.config')
model_dir = path_to_model

# Load pipeline config and build the detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
manager = tf.train.CheckpointManager(ckpt, path_to_model, max_to_keep=1)
status = ckpt.restore(manager.latest_checkpoint).expect_partial()

In [4]:
@tf.function
def detect_fn(image):
    image = tf.cast(image, tf.float32)/255.
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    
    return detections

# Save the detection model as SavedModel format
saved_model_path = 'saved_model'
tf.saved_model.save(detection_model, saved_model_path, signatures=detect_fn.get_concrete_function(tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.uint8)))

Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [5]:
loaded_model = tf.saved_model.load(saved_model_path)
infer = loaded_model.signatures["serving_default"]

In [6]:
import cv2
img = cv2.imread("../img.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(799,504))

In [7]:
detections = infer(tf.expand_dims(tf.constant(img), axis=0))

2023-10-10 17:07:49.215250: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected a subtype of type_id: TFT_TENSOR
args {
  type_id: TFT_INT32
}
 for input 2 of a homogeneous container 1001, got type_id: TFT_SHAPE_TENSOR
args {
  type_id: TFT_INT32
}

	while inferring type of node 'StatefulPartitionedCall/Preprocessor/map/while/body/_1106/Preprocessor/map/while/TensorArrayV2Write_1/TensorListSetItem'
2023-10-10 17:07:49.236211: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
'''
'raw_detection_boxes'
'detection_multiclass_scores'
'detection_classes'
'detection_boxes'
'raw_detection_scores'
'num_detections'
'detection_anchor_indices'
'detection_scores'
'''

detections.keys()

dict_keys(['raw_detection_boxes', 'detection_multiclass_scores', 'detection_classes', 'detection_boxes', 'raw_detection_scores', 'num_detections', 'detection_anchor_indices', 'detection_scores'])

In [9]:
formatted_img = img
formatted_img = tf.cast(formatted_img, tf.float32)/255.

detections_ckpt = detection_model(tf.expand_dims(tf.constant(formatted_img), axis=0))

In [10]:
detections_ckpt['detection_scores']

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[0.24635486, 0.22270001, 0.22234909, 0.21281295, 0.2101372 ,
        0.20808034, 0.19575262, 0.19410048, 0.1892952 , 0.1866749 ,
        0.17956518, 0.17705466, 0.17646493, 0.17130014, 0.16175069,
        0.16092595, 0.1593827 , 0.15827537, 0.15793754, 0.15778166,
        0.1552761 , 0.15154712, 0.15037756, 0.1502724 , 0.14940484,
        0.14929412, 0.14804104, 0.14708124, 0.14704692, 0.1462922 ,
        0.14548752, 0.14524597, 0.14467156, 0.14464808, 0.14454292,
        0.14404467, 0.14393423, 0.1430473 , 0.14244148, 0.14206186,
        0.14197952, 0.14192335, 0.1416538 , 0.14154382, 0.14145419,
        0.14125241, 0.13960525, 0.13934524, 0.13923217, 0.13923156,
        0.13902617, 0.1389157 , 0.13760494, 0.13575944, 0.13532606,
        0.13488081, 0.13486369, 0.13484769, 0.13483155, 0.13480808,
        0.13454688, 0.13452736, 0.13429573, 0.13399595, 0.13315977,
        0.13258882, 0.13241027, 0.1323258 , 0.13203074, 0.13145734

In [11]:
detections['detection_scores']

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[0.24635597, 0.22270034, 0.22234887, 0.21281272, 0.21013775,
        0.20808026, 0.19575277, 0.19410044, 0.18929596, 0.18667574,
        0.17956501, 0.1770551 , 0.17646493, 0.17130043, 0.1617511 ,
        0.16092603, 0.15938303, 0.15827456, 0.15793797, 0.15778163,
        0.15527628, 0.15154642, 0.15037681, 0.15027246, 0.1494049 ,
        0.14929432, 0.14804104, 0.14708112, 0.14704709, 0.14629267,
        0.14548774, 0.14524682, 0.14467153, 0.1446482 , 0.14454292,
        0.14404455, 0.14393497, 0.14304748, 0.14244163, 0.14206226,
        0.1419798 , 0.14192379, 0.14165406, 0.1415437 , 0.14145456,
        0.14125246, 0.13960539, 0.13934594, 0.13923217, 0.1392315 ,
        0.13902576, 0.13891596, 0.13760448, 0.1357591 , 0.13532606,
        0.13488102, 0.13486397, 0.13484772, 0.1348311 , 0.13480835,
        0.13454713, 0.134528  , 0.13429637, 0.13399595, 0.13315992,
        0.13258846, 0.13240992, 0.13232605, 0.13203022, 0.13145751